In [3]:
import duckdb
import pandas as pd
import xml.etree.ElementTree as ET

C:\Users\Lione\AppData\Local\Temp\ipykernel_7804\4110808938.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
#tabla paises 
pais = duckdb.sql('''SELECT name AS Nombre FROM enunciado_paises.csv''')
df_pais = pais.df()
df_pais

,Nombre
0,Belgium
1,England
2,France
3,Germany
4,Italy
5,Netherlands
6,Poland
7,Portugal
8,Scotland
9,Spain


In [4]:
#tabla jugador
jugador = duckdb.sql('''SELECT player_api_id AS ID, player_name AS Nombre FROM enunciado_jugadores.csv''')
df_jugador = jugador.df()
df_jugador

,ID,Nombre
0,505942,Aaron Appindangoye
1,155782,Aaron Cresswell
2,162549,Aaron Doran
3,30572,Aaron Galindo
4,23780,Aaron Hughes
...,...,...
11055,26357,Zoumana Camara
11056,111182,Zsolt Laczko
11057,36491,Zsolt Low
11058,35506,Zurab Khizanishvili


In [4]:
#tabla temporada, el id fue creado con pandas
temporada = duckdb.sql('''SELECT DISTINCT season AS Temporada FROM enunciado_partidos.csv''')
df_temporada = temporada.df()         # paso el objeto duckdb a df de pandas
df_temporada['ID'] = range(len(df_temporada),0,-1) # para cada valor de temporada en la nueva columna pongo ese id
df_temporada.to_csv('tabla_temporada', index=False)

In [5]:
# tabla liga, EL ID ESTA MAL, debería ser uno de estos [1,  1729,  4769,  7809, 10257, 13274, 15722, 17642, 19694,21518, 24558]
# pero como saber cual es cual? YA SE, gracias al country_id del partido
liga = duckdb.sql('''
                  SELECT DISTINCT l.name AS Nombre_liga, p.name AS Nombre_pais, part.league_id AS ID
                  FROM enunciado_liga.csv AS l
                  INNER JOIN enunciado_paises.csv AS p ON l.country_id = p.id
                  INNER JOIN enunciado_partidos.csv AS part ON l.country_id = part.country_id
                  ''')

df_liga = liga.df()
df_liga


,Nombre_liga,Nombre_pais,ID
0,Netherlands Eredivisie,Netherlands,13274
1,Belgium Jupiler League,Belgium,1
2,Spain LIGA BBVA,Spain,21518
3,Scotland Premier League,Scotland,19694
4,England Premier League,England,1729
5,Italy Serie A,Italy,10257
6,Portugal Liga ZON Sagres,Portugal,17642
7,France Ligue 1,France,4769
8,Germany 1. Bundesliga,Germany,7809
9,Switzerland Super League,Switzerland,24558


In [3]:
# Tabla Equipos
equipo = duckdb.sql('''
                    SELECT DISTINCT eq.team_api_id AS ID_equipo, eq.team_long_name AS Nombre, par.league_id AS ID_liga
                    FROM enunciado_equipos.csv AS eq
                    INNER JOIN enunciado_partidos.csv AS par ON eq.team_api_id = par.home_team_api_id
                    ''')

df_equipo = equipo.df()
#EXPLICACION: basicamente hice el join en base a la liga donde ese equipo jugó de local, podria pasar que no hay jugado de local?
# según lo que estuve viendo no, porque me unió a todos los equipos
df_equipo.to_csv('tabla_equipo', index=False)


In [ ]:
goleador = duckdb.sql('''
                      SELECT DISTINCT match_api_id AS ID_partido, goal AS Goles
                      FROM enunciado_partidos.csv
                      ''')

df_goleador = goleador.df().dropna(subset=['Goles'])
goles = df_goleador["Goles"]
#print(f"Hay {len(goles)} filas")
columna_final = []

for gol in goles:
    # Parsear el string XML
    root = ET.fromstring(gol)
    # Función para convertir XML a diccionario
    def xml_to_dict(element):
        result = {}
        if element.tag == "goal":
            result["values"] = []  # Inicializa la lista de valores
            for value in element.findall('value'):
                value_dict = xml_to_dict(value)  # Convierte cada valor a dict
                result["values"].append(value_dict)  # Agrega el dict a la lista
        else:
            for child in element:
                # Llamar recursivamente si hay más hijos
                result[child.tag] = xml_to_dict(child) if len(child) > 0 else child.text
        return result
    # Convertir el XML a diccionario
    goal_dict = xml_to_dict(root)
    # Appendeamos
    columna_final.append(goal_dict)
    



In [ ]:
for goles_en_partido in columna_final:
    for gol in goles_en_partido['values']:

        print(gol.keys())
        gol['player1'] # id_jugador
        print(gol['stats'])
        if gol['stats']['goals'] != 1:
            print(f"Hay {gol['stats']['goals']} goles en una llamada.")

In [28]:
# Tabla Partidos
partidos = duckdb.sql('''
                      SELECT DISTINCT match_api_id AS ID_partido, date AS Fecha, home_team_api_id AS ID_local, away_team_api_id AS ID_visitante, home_team_goal AS Goles_local, away_team_goal AS Goles_visitante,
                      CASE WHEN home_team_goal > away_team_goal THEN 'Ganado'
                      WHEN away_team_goal > home_team_goal THEN 'Perdido'
                      ELSE 'Empatado'
                      END AS Resultado,
                      FROM enunciado_partidos.csv, plantel.csv
                      ''')
partidos = partidos.df()

def fecha_a_temporada(fecha):
    if fecha < pd.to_datetime('2008-08-01'):
        return '2007/2008'
    if fecha < pd.to_datetime('2009-08-01'):
        return '2008/2009'
    if fecha < pd.to_datetime('2010-08-01'):
        return '2009/2010'
    if fecha < pd.to_datetime('2011-08-01'):
        return '2010/2011'
    if fecha < pd.to_datetime('2012-08-01'):
        return '2011/2012'
    if fecha < pd.to_datetime('2013-08-01'):
        return '2012/2013'
    if fecha < pd.to_datetime('2014-08-01'):
        return '2013/2014'
    if fecha < pd.to_datetime('2015-08-01'):
        return '2014/2015'
    if fecha < pd.to_datetime('2016-08-01'):
        return '2015/2016'
    else: 
        return 'ERROR 404'

partidos['Temporada'] = partidos.Fecha.apply(fecha_a_temporada)
partidos.to_csv('tabla_partidos.csv', index =False)

In [37]:
# Consulta para chequear que todos los equipos juegan de local
todos_juegan_local = duckdb.sql('''
                                SELECT ID_equipo
                                FROM tabla_equipo.csv
                                EXCEPT
                                SELECT ID_local
                                FROM tabla_partidos.csv
                                ''')
si = todos_juegan_local.df()
si

,algo


In [39]:
#Tabla Plantel
# Acá hice un cross join, suponiendo que todos los equipos jugaron todas las temporadas... FALSO
plantel = duckdb.sql('''
                     SELECT DISTINCT Temporada, ID_local AS ID_equipo
                     FROM tabla_partidos.csv 
                     ''')

df_plantel = plantel.df()
df_plantel['ID_plantel'] = range(0,len(df_plantel),1)
df_plantel.to_csv('tabla_plantel.csv', index=False)

In [15]:
# Conformación plantel
conf_plantel = duckdb.sql('''
                          SELECT DISTINCT ID_plantel, home_player_1 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_2 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_3 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_4 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_5 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_6 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_7 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_8 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_9 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_10 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, home_player_11 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND home_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_1 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_2 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_3 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_4 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_5 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_6 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_7 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_8 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_9 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_10 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          UNION
                          SELECT DISTINCT ID_plantel, away_player_11 AS ID_jugador
                          FROM enunciado_partidos.csv, plantel.csv
                          WHERE season = Temporada AND away_team_api_id = ID_equipo
                          ''')

# una vez que hago todos estos simplemente hago un join y listo
conformacion_plantel = conf_plantel.df()
conformacion_plantel = conformacion_plantel.dropna(subset=['ID_jugador']) # acá borré los nulls para poder aplicar la funcion para sacar ese id.0 que aparece en todos los jugadores de partidos
conformacion_plantel['ID_jugador'] = conformacion_plantel['ID_jugador'].apply(lambda x: int(x)) # saco esos .0
conformacion_plantel.to_csv('tabla_conformacion_plantel.csv',index=False)


# agarro la tabla plantel y digo: si en partidos coincide el season con id_temporada Y local con equipo SELECT id_plantel con id_home player